In [1]:
from prob_4e import *  # Temp for now. I want to check sampling inference algos before I move all this in main file.

In this brief tutorial, we will try to understand how variable elimination is implemented in the probability module. To help us with this we will take an example and simultaneously work it out by hand and draw comparisons with our code implementation.

To begin with we need a simple bayesian network and an inference problem. We use the one explained below.

Rain -> Traffic -> Late

Rain can cause traffic which can result in us getting late. The distribution of the nodes is given by.

| Rain | P(Rain)     |
|------|-------------|
| T   | 0.1         |
| F   | 0.9         |

| Rain | Traffic | P(Traffic &#124; Rain) |
|----|----|-----------------|
| T | T | 0.8 |
| T | F | 0.2 |
| F | T | 0.1 |
| F | F | 0.9 |

| Traffic | Late | P(Late &#124; Traffic) |
|----|----|-----------------|
| T | T | 0.3 |
| T | F | 0.7 |
| F | T | 0.1 |
| F | F | 0.9 |

We can create a Bayesian network using **BayesNet** Class and then invoke the **add** method to add each of our nodes. You can refer to the Probability Notebook to find out how to create a Bayesian Network.

Let us assume we want to solve the problem of finding out the **P(Late)**.

In [2]:
traffic_net = (BayesNet()
    .add('Rain', [], 0.1)
    .add('Traffic', ['Rain'], {T: 0.8, F: 0.1})
    .add('Late', ['Traffic'], {T: 0.3, F: 0.1}))

Variable Elimination works on the concept of eliminating all hidden variables. We do this by joining and eliminating variables alternatively. This helps us to keep the size of the join in control.

In our problem, both Rain and Traffic are hidden variables.

In reading the Probability notebook you might have noticed that the way we store probabilities of our variables is slightly different from the tables above.

In [3]:
traffic_net.variables

[Rain, Traffic, Late]

Let us look at the cpt of the Traffic Variable.

In [4]:
Rain, Traffic, Late = traffic_net.variables
Traffic.cpt

{(F,): {F: 0.9, T: 0.1}, (T,): {F: 0.19999999999999996, T: 0.8}}

Here each key in the cpt is that of the value of the parent and it is mapped to a distribution for the variable itself. Therefore each row in the cpt corresponds to two in the corresponding table.

| Rain | Traffic | P(Traffic &#124; Rain) |
|----|----|-----------------|
| T | T | 0.8 |
| T | F | 0.2 |
| F | T | 0.1 |
| F | F | 0.9 |

The (F,) in the cpt corresponds to the parent Rain being False and then the nested {F: 0.9, T: 0.1} list the probabilities in the last two rows of the table. Our implementation of **Factor** results in mapping exactly similar to the mappings. The function **variable_to_factor** converts each variable cpt to corresponding factors.

In [5]:
%psource variable_to_factor

In [6]:
rain_factor, traffic_factor, late_factor = [variable_to_factor(var) for var in traffic_net.variables]

In [7]:
traffic_factor

{(F, F): 0.9, (F, T): 0.1, (T, F): 0.19999999999999996, (T, T): 0.8}

Now that the **traffic_factor** looks exactly the same as the table let us focus on solving the problem at hand. As per the Variable Elimination technique, we select a hidden variable and join all the factors that use it. Finally, we eliminate the hidden variable and carry on our process.

We will first try to eliminate Rain. The tables for factors containing Rain are given below.

| Rain | P(Rain)     |
|------|-------------|
| +r   | 0.1         |
| -r   | 0.9         |

| Rain |Traffic | P(Traffic &#124; Rain) |
|----|----|-----------------|
| T | T | 0.8 |
| T | F | 0.2 |
| F | T | 0.1 |
| F | F | 0.9 |

We should create a join of these of two variables as a first step to solving this problem. A join is very similar to a database join. It results in a new table. The columns in our new table will are any of the columns in either of the two tables i.e. the union. The rows will be the Cartesian Product of the domain of each of the columns.

So on joining we would get a table like because the union of variables is {Rain, Traffic}

| Rain |Traffic | f<sub>1</sub>(Traffic, Rain)  |
|----|----|-----------------|
| T | T | ? |
| T | F | ?|
| F | T | ? |
| F | F | ? |

We still need to compute what will be the probabilities in each of those rows. It can be given by multiplying the consistent rows. For example to compute the value for **Rain=F,Traffic=F** we look at the last row of both our tables and get 
**0.9 * 0.9 = 0.81**

You can verify the same for the complete table now.

| Rain |Traffic | f<sub>1</sub>(Traffic, Rain)  |
|----|----|-----------------|
| T | T | 0.08 |
| T | F | 0.02 |
| F | T | 0.09 |
| F | F | 0.81 |

The functionality to do the same operation using the **Factor** implementation is provided by the **pointwise_product** method. This method follows exactly same steps for calculating joins.

In [8]:
%psource Factor.pointwise_product

In [9]:
join_rain_traffic = rain_factor.pointwise_product(traffic_factor)
join_rain_traffic

{(F, F): 0.81,
 (F, T): 0.09000000000000001,
 (T, F): 0.019999999999999997,
 (T, T): 0.08000000000000002}

You can verify that the above matches well with the results we calculated by hand.

Now after the calculating the join, we need to finally get rid of Rain using another operation called summing-out or marginalization.

After this operation, we get a new table with the eliminated variable gone. The rows of this table are again the cartesian product of the domain of remaining variables.

|Traffic| f<sub>2</sub>(Traffic) |
|----|--------------|
| T | ? |
| F | ? |

To fill out the entries in this table we look at the corresponding entries in the old table and sum them together.

| Rain |Traffic | f<sub>1</sub>(Traffic, Rain)  |
|----|----|-----------------|
| T | T | 0.08 |
| T | F | 0.02 |
| F | T | 0.09 |
| F | F | 0.81 |

So to calculate the value for +t we see the corresponding entries Rain=T,Traffic=T and Rain=F,Traffic=T and sum them together to get 0.17. We repeat this for the rest of the rows in our new table.

|Traffic| f<sub>2</sub>(Traffic) |
|----|--------------|
| T | 0.17 |
| F | 0.83 |

The same operation can be performed by using the method **Factor.sum_out**. It follows the same series of steps described above.

In [10]:
%psource Factor.sum_out

In [11]:
r_summed_out = join_rain_traffic.sum_out(Rain)
r_summed_out

{(F,): 0.8300000000000001, (T,): 0.17000000000000004}

Now we can repeat the above procedure to join our latest factor and eliminate on T.

In [12]:
join_on_t = r_summed_out.pointwise_product(late_factor)
join_on_t

{(F, F): 0.7470000000000001,
 (F, T): 0.11900000000000002,
 (T, F): 0.08300000000000002,
 (T, T): 0.05100000000000001}

In [13]:
ans = join_on_t.sum_out(Traffic)
ans

{(F,): 0.8660000000000001, (T,): 0.13400000000000004}